In [ ]:
import requests
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, LineString, Polygon
import folium

Use API to fetch data

In [ ]:
# Define the API endpoint
url = "https://www.federalregister.gov/api/v1/documents.json"

# Define query parameters
params = {
    "conditions[publication_date][gte]": "2024-01-01",  # Start date
    "conditions[publication_date][lte]": "2024-12-31",  # End date
    "conditions[agencies][]": "indian-affairs-bureau",  # Filter by agency
    "conditions[type][]": "NOTICE",
    "conditions[term]": "Land Acquisition",  # Search term
    "fields[]": "raw_text_url",
    "order": "relevance",  # Sort order
    "per_page": 2  # Number of results per page
}

# Make the initial API request
response = requests.get(url, params=params)

# Check for errors
if response.status_code == 200:
    data = response.json()
    raw_text_urls = [doc["raw_text_url"] for doc in data.get("results", [])]
    print(f"Found {len(raw_text_urls)} documents.")
    
    # Process each raw text URL
    for raw_text_url in raw_text_urls:
        print(f"Fetching content from: {raw_text_url}")
        raw_response = requests.get(raw_text_url)
        
        if raw_response.status_code == 200:
            # Extract and print the raw text
            raw_text = raw_response.text
            print(f"Content Preview:\n{raw_text[:2000]}...\n")  # Print the first 1000 characters for preview
            print("-" * 80)
        else:
            print(f"Error fetching raw text: {raw_response.status_code}")
else:
    print(f"Error: {response.status_code} - {response.text}")


Load PLSS data for given section

In [ ]:
# Load the PLSS shapefile 
shapefile_path = '/Users/mbraaksma/Downloads/BLM_NATL_PLSS.gdb/ilmocplss.gdb'
div_gdf = gpd.read_file(shapefile_path, 
                    layer='PLSSIntersected', 
                    where="STATEABBR = 'MN' AND TWNSHPNO = '108' AND TWNSHPDIR = 'N' AND RANGENO = '015' AND RANGEDIR = 'W' AND FRSTDIVNO = '01'", 
                    engine='pyogrio')

Attempt to create polygon

In [ ]:
def calculate_new_point(start_point, bearing, distance):
    # Convert bearing to radians
    bearing_rad = np.deg2rad(bearing)
    
    # Distance to radians conversion (earth radius approx 6371000 meters)
    distance_rad = distance / 6371000.0

    lat1 = np.deg2rad(start_point.y)
    lon1 = np.deg2rad(start_point.x)

    lat2 = np.arcsin(np.sin(lat1) * np.cos(distance_rad) +
                     np.cos(lat1) * np.sin(distance_rad) * np.cos(bearing_rad))

    lon2 = lon1 + np.arctan2(np.sin(bearing_rad) * np.sin(distance_rad) * np.cos(lat1),
                             np.cos(distance_rad) - np.sin(lat1) * np.sin(lat2))

    return Point(np.rad2deg(lon2), np.rad2deg(lat2))

def create_curve(start_point, radius, central_angle, direction='concave southwest'):
    # Placeholder function for curve handling, to be developed as needed
    # This will require more advanced geometric calculations or interpolation
    # Return a list of points along the curve
    return [start_point]  # Dummy placeholder

# Get the northwest corner of the section
section_polygon = div_gdf.geometry.unary_union
northwest_corner = Point(section_polygon.bounds[0], section_polygon.bounds[3])

# Trace the described polygon
points = [northwest_corner]
# South 00 degrees 54 minutes 41 seconds East, 778.98 feet
points.append(calculate_new_point(points[-1], 180 + 0.9114, 778.98 * 0.3048))
# South 44 degrees 55 minutes 49 seconds East, 764.84 feet
points.append(calculate_new_point(points[-1], 180 + 44.9303, 764.84 * 0.3048))
# Start of new line, another point at South 44 degrees 55 minutes 49 seconds East, 5121.99 feet
points.append(calculate_new_point(points[-1], 180 + 44.9303, 5121.99 * 0.3048))
# Curve segment with radius 1083.65 feet and central angle 20 degrees 34 minutes 11 seconds
curve_points = create_curve(points[-1], 1083.65 * 0.3048, 20 + 34/60 + 11/3600)
points.extend(curve_points)

# Close the polygon if needed
# points.append(points[0])

# Create a polygon from points
polygon = Polygon(points)

# Visualize or save the new polygon
gdf_polygon = gpd.GeoDataFrame(geometry=[polygon], crs=div_gdf.crs)


Plot

In [ ]:
# Plot
m = div_gdf.drop(columns=['SOURCEDATE', 'REVISEDDATE']).explore(name="Section")
m = gdf_polygon.explore(m=m, color="red", name="Parcel")
# this is completely optional
folium.LayerControl().add_to(m)

m